# Описание

Выполнил небольшой проект для проверки усвоения пройденного материала

Наборы данных:
- visits.csv
- cart.csv
- checkout.csv
- purchase.csv

In [4]:
import pandas as pd

visits = pd.read_csv('../files/visits.csv', parse_dates=[1])
# параметр parse_dates с указанием индекса колонки в которой хранится дата значительн опрощает процесс переобозначения типа данных
cart = pd.read_csv('../files/cart.csv', parse_dates=[1])
checkout = pd.read_csv('../files/checkout.csv', parse_dates=[1])
purchase = pd.read_csv('../files/purchase.csv', parse_dates=[1])

print(visits.head(), cart.head(), checkout.head(), purchase.head(), sep='\n\n')

                                user_id          visit_time
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00
3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00
4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00

                                user_id           cart_time
0  2be90e7c-9cca-44e0-bcc5-124b945ff168 2017-11-07 20:45:00
1  4397f73f-1da3-4ab3-91af-762792e25973 2017-05-27 01:35:00
2  a9db3d4b-0a0a-4398-a55a-ebb2c7adf663 2017-03-04 10:38:00
3  b594862a-36c5-47d5-b818-6e9512b939b3 2017-09-27 08:22:00
4  a68a16e2-94f0-4ce8-8ce3-784af0bbb974 2017-07-26 15:48:00

                                user_id       checkout_time
0  d33bdc47-4afa-45bc-b4e4-dbe948e34c0d 2017-06-25 09:29:00
1  4ac186f0-9954-4fea-8a27-c081e428e34e 2017-04-07 20:11:00
2  3c9c78a7-124a-4b77-8d2e-e1926e011e7d 2017-07-13 11:38:00
3  89fe330a-8966-4756-8f7c-3bdbcd47279

In [9]:
# объединить visits и cart левым соединением
visits_cart_left = pd.merge(visits, cart, how='left')

print(visits_cart_left.head())

                                user_id          visit_time  \
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00   
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00   
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00   
3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00   
4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00   

            cart_time  
0                 NaT  
1 2017-01-26 14:44:00  
2 2017-08-20 08:31:00  
3                 NaT  
4                 NaT  


In [10]:
# получите длинну нового набора
print(len(visits_cart_left))

2000


In [11]:
# сколько не сопоставленных значений из набора cart?
print(len(visits_cart_left[visits_cart_left.cart_time.isna()]))

1652


In [13]:
# какой процент пользователе посетивших веб магазин положили футболку в корзину ?
print(round((len(visits_cart_left) - len(visits_cart_left[visits_cart_left.cart_time.isna()]))*100/len(visits_cart_left), 2), '%')

17.4 %


In [14]:
# повторите левое соединения для наборов cart и checkout и посчитайте кол-во несопоставленных строк
cart_checkout_left = pd.merge(cart, checkout, how='left')

print(len(cart_checkout_left[cart_checkout_left.checkout_time.isna()]))

122


In [16]:
# объедините все наборы в левым соединением в переменную all_data последовательность из методов merge()
# последовательность visits - cart - checkout - purchase
all_data = visits.merge(cart, how='left').merge(checkout, how='left').merge(purchase, how='left').reset_index(drop=True)

print(all_data.head())

                                user_id          visit_time  \
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00   
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00   
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00   
3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00   
4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00   

            cart_time       checkout_time       purchase_time  
0                 NaT                 NaT                 NaT  
1 2017-01-26 14:44:00 2017-01-26 14:54:00 2017-01-26 15:08:00  
2 2017-08-20 08:31:00                 NaT                 NaT  
3                 NaT                 NaT                 NaT  
4                 NaT                 NaT                 NaT  


In [18]:
# Посчитайте процент пользователей которые подтвердили покупку но не купили футболку
checkout_but_not_purchase = all_data[all_data.checkout_time.notna() & all_data.purchase_time.isna()]

print(round(len(checkout_but_not_purchase)*100/len(all_data), 2), '%')

4.26 %


In [19]:
# добавьте новую колонку в которой будет разница между временем покупки и временем посещения сайта назовите колонку delta
all_data['delta'] = all_data.purchase_time - all_data.visit_time

print(all_data.head())

# получите среднее время delta
print(all_data.delta.mean())

                                user_id          visit_time  \
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00   
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00   
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00   
3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00   
4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00   

            cart_time       checkout_time       purchase_time           delta  
0                 NaT                 NaT                 NaT             NaT  
1 2017-01-26 14:44:00 2017-01-26 14:54:00 2017-01-26 15:08:00 0 days 00:44:00  
2 2017-08-20 08:31:00                 NaT                 NaT             NaT  
3                 NaT                 NaT                 NaT             NaT  
4                 NaT                 NaT                 NaT             NaT  
0 days 00:43:53.360160965
